## 1.模型构造

### 1.1 继承Module类来构造模型

* 可以基于``Sequential类``构造模型
* 也可以基于``Module类``构造模型

``Module``类是``nn``模块里提供的一个模型构造类,是所有神经网络结构的``基类``.通过继承这个基类来定义我们想要的模型.

这里定义的MLP类``重载``了Module类的``__init__``函数和``forward``函数.分别用于``初始化模型参数``和定义``前向计算``.

In [1]:
import torch
from torch import nn

In [2]:
class MLP(nn.Module):
    def __init__(self, **kwargs):
        # 调用MLP父类Module的构造函数来进行必要的初始化.
        # 使用**kwargs定义参数时，kwargs将会接收一个positional argument后所有关键词参数的字典。
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10) # 输出层
    
    # 定义前向计算
    # 目的是根据输入x计算模型输出.
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)
    

MLP类中无需定义反向传播函数.系统通过自动求梯度而``自动生成``反向传播所需的``backward``函数.

可通过``实例化MLP类``得到模型变量net.  
net(x)会调用继承自Module类的__call__函数,这个函数将调用MLP类定义的forward函数完成前向计算.  

也即,实例化之后,MLP类中的``前向计算会自动完成``.

In [3]:
X = torch.rand(2, 784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0675, -0.1139, -0.0486, -0.2151, -0.0018, -0.0563, -0.0357,  0.1635,
         -0.2710,  0.0487],
        [-0.1617,  0.0408, -0.0064, -0.0279, -0.0036, -0.0329,  0.0648,  0.1680,
         -0.1194,  0.2270]], grad_fn=<AddmmBackward>)

Module类是一个可供自由组建的部件.它的子类既可以是一个层(如Linear类),又可以是一个模型(如上面的MLP类),或者是模型的一个部分.

### 1.2 Module的子类

继承自Module类的一些可以方便的构造模型的类还有:  
* Sequential类  
* ModuleList类  
* ModuleDict类

1)Sequential类  
以更简单的方式定义模型．它``接受``一个子模块的``有序字典``，或者，一系列子模块作为参数逐一添加``Ｍodule的实例``.在前向计算时就是将这些实例按添加顺序逐一计算.

2)ModuleList类  
``接收``一个``子模块的列表``作为输入,然后也可以类似List那样进行append和extend操作.  

In [4]:
# net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
# net.append(nn.Linear(256, 10)) # # 类似List的append操作
# print(net[-1])  # 类似List的索引访问
# print(net)


#### Question:  
a) ``Sequential``和``MoudleList``的区别?  

ModuleList仅仅储存各种模块的列表,模块之间没有联系,没有顺序,而且没有实现forward.  
Sequential内的模块需要按照顺序排列,且要保证邻层的输入输出大小相匹配,内部forward功能已经实现.  

b) ``ModuleList``的出现有什么作用?  

让网络定义前向传播时更加灵活.  
另外,加入到ModuleList中的模块会自动添加到网络中,看下面代码中的例子对比.

In [5]:
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList, self).__init__()
        # 加入到ModuleList中的模块自动添加到网络中.
        self.linears = nn.ModuleList([nn.Linear(10, 10)])

class Module_List(nn.Module):
    def __init__(self):
        super(Module_List, self).__init__()
        # 这个线性层没有出现在网络结构中.
        self.linears = [nn.Linear(10, 10)]

net1 = Module_ModuleList()
net2 = Module_List()

print("net1:")
for p in net1.parameters():
    print(p.size())

print("net2:")
for p in net2.parameters():
    print(p)

print(net1)
print("=========")
print(net2)


net1:
torch.Size([10, 10])
torch.Size([10])
net2:
Module_ModuleList(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
  )
)
Module_List()


3)ModuleDict类  
``接受``一个子模块的``字典``作为输入,然后可以类似字典那样进行添加和访问操作.  
与ModuleList一样,仅仅存放了一个字典,forward函数需要自己实现,同时,ModuleDict里的模块也会自动添加到整个网络中.

In [6]:
# 以字典作为输入
net = nn.ModuleDict({
    'Linear':nn.Linear(784, 256),
    'act':nn.ReLU(),
})
# 添加
net['output'] = nn.Linear(256, 10)
print(net)

ModuleDict(
  (Linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


### 1.3 构造复杂的模型

#### 一个复杂一点的``FancyMLP``网络.  
* 通过``get_constant``函数创建训练中不被迭代的参数,即``常数参数``.
* 前向计算中,使用``Tensor``函数和Python的``控制流``,并多次``调用相同的层``.

In [20]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)

        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数(常数参数)
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)

        # 使用创建的常数参数,以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)

        # 复用全连接层.等价于两个全连接层共享参数
        x = self.linear(x)

        # print("++++")
        # print(x.shape) #(2, 20)
        # print(x.norm())
        # print(x.norm().item())
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)


# 从结果看出,网络结构中只定义了一个linear层,但是前向计算中可以对linear层进行多次调用.




FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(3.9904, grad_fn=<SumBackward0>)

#### 定义多个继承自nn.Module的模块还可以作为Sequential的输入.

* 继承自Module类的模块可以作为``模型的一部分``,每个部分定义了``各自的前向计算过程``.每个部分可以作为Sequential的输入组合到一起,构成整个网络.

* 印证前文所述:继承自Module类的子类是一个可供自由组建的部件.它可以是一个层,可以是一个模型,也可以是模型的一部分

In [21]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU())
    
    def forward(self, x):
        return self.net(x)
    
net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)


Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-0.4560, grad_fn=<SumBackward0>)

## 2.模型参数的访问,初始化与共享

## 3.模型参数的延后初始化

## 4.自定义层